In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Soumyadipta\\Desktop\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [16]:
%pwd

'c:\\Users\\Soumyadipta\\Desktop\\Chicken-Disease-Classification'

In [17]:
from dataclasses import dataclass
from pathlib import Path

In [18]:
@dataclass(frozen=True)
class DataIngestionConfig: # entity name is DataIngestionConfig
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info : \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function retrns None
        """   
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-28 20:06:41,152: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-28 20:06:41,155: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-28 20:06:41,157: INFO: common: created directory at: artifacts]
[2024-02-28 20:06:41,159: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-28 20:06:45,225: INFO: 366118637: artifacts/data_ingestion/data.zip downloaded with the following info : 
Connection: close
Content-Length: 11595077
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "610dd053ec8fbeb9803ceabd7ed103c59f22336d3578de69daa7fd818acde3e0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 964C:18E4D6:5EBDB1:7A52E6:65DF44FA
Accept-Ranges: bytes
Date: Wed, 28 Feb 2024 14:36:44 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10237-MAA
X-Ca

In [ ]:
# Now we can convert this code into modular coding.